**2025 Military AI CONtest**<br>
AI Autonomous Driving – Pre-Training Code Sample<br> 
NVIDIA Jetson Nano/Wingbot/JupyterLab<br>
Version : v1.0<br>
File : robot_line_following_test.ipynb<br>
**노란선 사이 흰색선 추종 테스트 코드**


**1. 라이브러리 및 초기화**<br>


In [ ]:
import cv2
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import time
import subprocess
import warnings
import os
from tiki.mini import TikiMini

# GStreamer 경고 메시지 필터링
# "Cannot query video position" 경고는 실시간 카메라 스트림에서 정상적인 메시지입니다.
# 비디오 파일이 아닌 실시간 스트림은 위치 정보가 없기 때문에 나타나는 경고입니다.
# 이 경고는 무시해도 되며, 카메라 기능에는 영향을 주지 않습니다.
warnings.filterwarnings('ignore', category=UserWarning)
# GStreamer 디버그 메시지 최소화 (필요시 '0'으로 설정하여 에러만 표시)
os.environ['GST_DEBUG'] = '0'  # 0=에러만, 1=경고, 2=정보

# 로봇 초기화
tiki = TikiMini()

# 모터 모드 설정 (PWM 모드)
tiki.set_motor_mode(tiki.MOTOR_MODE_PWM)

print("로봇 초기화 완료")

# ============================================
# 전역 상수 및 변수 정의
# ============================================

# 디버그 설정
DETECT_WHITE_BETWEEN_YELLOW_DEBUG = False  # True로 설정하면 상세 로그 출력
DETECT_WHITE_BETWEEN_YELLOW_SHOW_IMAGES = False  # True로 설정하면 중간 이미지 표시
DETECT_WHITE_LINE_DEBUG = False  # True로 설정하면 상세 로그 출력
DETECT_WHITE_LINE_SHOW_IMAGES = False  # True로 설정하면 중간 이미지 표시

# 주행 파라미터
base_speed = 25  # 기본 속도 (0-127)
max_steering = 20  # 최대 조향값
frame_center_x = 160  # 프레임 중앙 X 좌표 (320/2)

# 주행 상태 추적
driving_state = "STRAIGHT"  # STRAIGHT, CURVE, INTERSECTION
error_history = []  # 오차 히스토리 (커브 감지용)
max_history = 10  # 최대 히스토리 길이

# 색상 검출 파라미터
yellow_lower = [20, 100, 100]  # 노란색 하한 HSV 값
yellow_upper = [30, 255, 255]  # 노란색 상한 HSV 값
white_threshold = 180  # 흰색 임계값 (조명 환경에 따라 150-220 조정)

# 카메라 파이프라인 설정 (여러 옵션 시도)
pipeline_options = [
    # 옵션 1: 기본 파이프라인
    (
        "nvarguscamerasrc ! video/x-raw(memory:NVMM), width=320, height=240, format=NV12, framerate=30/1 ! "
        "nvvidconv ! video/x-raw, format=BGRx ! videoconvert ! video/x-raw, format=BGR ! appsink"
    ),
    # 옵션 2: 낮은 프레임레이트
    (
        "nvarguscamerasrc ! video/x-raw(memory:NVMM), width=320, height=240, format=NV12, framerate=15/1 ! "
        "nvvidconv ! video/x-raw, format=BGRx ! videoconvert ! video/x-raw, format=BGR ! appsink"
    ),
    # 옵션 3: 추가 옵션 없이
    (
        "nvarguscamerasrc ! video/x-raw(memory:NVMM), width=320, height=240, format=NV12 ! "
        "nvvidconv ! video/x-raw, format=BGRx ! videoconvert ! video/x-raw, format=BGR ! appsink"
    ),
]

cap = None
pipeline_used = None

# 카메라 초기화 시도
print("카메라 초기화 시도 중...")

# 먼저 다른 프로세스가 카메라를 사용 중인지 확인
try:
    result = subprocess.run(['fuser', '/dev/video0'], capture_output=True, text=True, timeout=1)
    if result.returncode == 0:
        print("[경고] 카메라가 다른 프로세스에서 사용 중일 수 있습니다.")
except:
    pass

# 각 파이프라인 옵션 시도
for i, pipeline in enumerate(pipeline_options, 1):
    print(f"파이프라인 옵션 {i} 시도 중...")
    cap = cv2.VideoCapture(pipeline, cv2.CAP_GSTREAMER)
    
    # 카메라가 열릴 때까지 잠시 대기
    time.sleep(0.5)
    
    if cap.isOpened():
        # 테스트 프레임 읽기
        ret, test_frame = cap.read()
        if ret:
            print(f"✓ 카메라 연결 성공! (파이프라인 옵션 {i} 사용)")
            pipeline_used = i
            break
        else:
            print(f"파이프라인 옵션 {i}: 프레임 읽기 실패")
            cap.release()
            cap = None
    else:
        print(f"파이프라인 옵션 {i}: 카메라 열기 실패")
        if cap:
            cap.release()
            cap = None

if cap is None or not cap.isOpened():
    print("\n[에러] 카메라 초기화 실패")
    print("가능한 해결 방법:")
    print("1. 다른 Jupyter 노트북이나 프로세스가 카메라를 사용 중인지 확인")
    print("2. Jetson Nano를 재부팅")
    print("3. 카메라 하드웨어 연결 확인")
    print("4. 권한 확인: sudo usermod -a -G video $USER")
    raise RuntimeError("카메라를 열 수 없습니다. 위의 해결 방법을 확인하세요.")

# 비디오 위젯 설정 (반드시 display() 전에 설정)
video_widget = widgets.Image(format='jpeg', layout=widgets.Layout(width='320px', height='240px'))

# 비디오 위젯 표시 (display는 초기화 후 한 번만 호출)
display(video_widget)

# 비디오 위젯이 제대로 표시되었는지 확인
print("비디오 위젯 표시 완료")
print(f"비디오 위젯 타입: {type(video_widget)}")
print("초기화 완료")


In [ ]:
# ============================================
# (구) 함수 정의 셀 - 현재는 아래 섹션(2-1, 2-2, 3)의 최신 함수들을 사용합니다.
# 필요하다면 이 셀에 공통 유틸 함수를 추가해서 사용하세요.
# ============================================

# 예시: 공통 유틸 함수 (필요시 추가)
# def frame_to_bytes(frame):
#     """프레임을 JPEG 바이트로 변환"""
#     _, buf = cv2.imencode('.jpg', frame, [int(cv2.IMWRITE_JPEG_QUALITY), 40])
#     return buf.tobytes()

print("이전 버전 함수 정의 셀입니다. 실제 사용 함수는 2-1, 2-2, 3 섹션 셀을 참고하세요.")

**2-0. 노란선 사이 흰색선 검출 코드 분석**<br>
<br>
**검출 알고리즘 단계별 분석:**<br>
1. **ROI 설정**: 프레임의 하단 40% 영역만 처리 (차선 인식에 적합)<br>
2. **노란선 검출**: HSV 색공간에서 노란색 검출<br>
   - 노란색 HSV 범위: H=20-30, S=100-255, V=100-255<br>
3. **양쪽 노란선 위치 찾기**: 좌측/우측 노란선의 중심 X 좌표 계산<br>
4. **검색 영역 정의**: 양쪽 노란선 사이의 영역 설정<br>
5. **흰색선 검출**: 정의된 영역 내에서 흰색선 검출 (임계값: 180 이상)<br>
6. **중심 계산**: 검출된 흰색선의 중심 X 좌표 계산<br>
<br>
**파라미터:**<br>
- **노란색 HSV 범위**:<br>
  - `yellow_lower = [20, 100, 100]`: 노란색 하한<br>
  - `yellow_upper = [30, 255, 255]`: 노란색 상한<br>
- **흰색 임계값**: `white_threshold = 180` (조명 환경에 따라 150-220 조정)<br>
- **ROI**: 하단 40%, 좌우 10-75% (경로에 따라 조정 가능)<br>
<br>
**주행 전략:**<br>
- 양쪽 노란선 사이의 흰색선을 따라 주행<br>
- 노란선이 검출되지 않으면 기존 ROI 영역에서 흰색선 검출<br>
<br>
**주행 모드 자동 감지:**<br>
1. **직진 (STRAIGHT)**: 오차가 작을 때 (±15픽셀 이내)<br>
2. **커브 (CURVE)**: 오차가 클 때 (±15픽셀 초과)<br>
3. **교차로 (INTERSECTION)**: 흰색선이 넓게 분포할 때 (X 범위가 검색 영역의 60% 이상)<br>
   - 교차로에서는 조향 없이 직진 유지<br>


**2-1. 노란선 사이 흰색선 검출 디버그 테스트 (선택사항)**<br>
<br>
**노란선과 흰색선 검출 함수를 테스트하고 로그를 확인하세요**<br>


In [ ]:
# 회색선 검출 함수 테스트 (디버그 모드)
# 카메라에서 프레임을 읽어 회색선 검출 과정을 단계별로 확인

if cap is None or not cap.isOpened():
    print("[에러] 카메라가 초기화되지 않았습니다. 먼저 초기화 셀을 실행하세요.")
else:
    print("노란선 사이 흰색선 검출 테스트 시작...")
    print("=" * 60)
    
    # 몇 프레임 테스트
    for test_i in range(5):
        ret, test_frame = cap.read()
        if ret:
            test_frame = cv2.flip(test_frame, -1)
            print(f"\n[테스트 프레임 {test_i+1}]")
            print("-" * 60)
            
            # 디버그 모드로 검출
            result = detect_white_line_between_yellow(test_frame, debug=True, show_images=False)
            
            # 반환값 길이에 따라 언팩 (메인 루프와 동일한 방식)
            if len(result) == 5:
                line_center_x, mask, roi, debug_info, is_intersection = result
            elif len(result) == 4:
                line_center_x, mask, roi, debug_info = result
                is_intersection = False
            else:
                line_center_x, mask, roi = result
                debug_info = None
                is_intersection = False
            
            if line_center_x is not None:
                print(f"✓ 라인 검출 성공: 중심 X = {line_center_x}")
                if is_intersection:
                    print("  - [교차로] 넓은 흰색 구간 감지")
            else:
                print("✗ 라인 검출 실패")
            
            if debug_info:
                print(f"\n[디버그 정보 요약]")
                print(f"  - 마스크 픽셀 비율: {debug_info.get('white', {}).get('percentage', 0):.2f}%")
                if 'detection' in debug_info and 'center_mode' in debug_info['detection']:
                    print(f"  - 중심 계산 모드: {debug_info['detection']['center_mode']}")
            
            time.sleep(0.5)
        else:
            print(f"프레임 {test_i+1} 읽기 실패")
    
    print("\n" + "=" * 60)
    print("테스트 완료")


**2-1. 노란선 사이 흰색선 검출 함수**<br>


In [ ]:
# 노란선 사이 흰색선 검출 디버그 설정
DETECT_WHITE_BETWEEN_YELLOW_DEBUG = False  # True로 설정하면 상세 로그 출력
DETECT_WHITE_BETWEEN_YELLOW_SHOW_IMAGES = False  # True로 설정하면 중간 이미지 표시

def detect_white_line_between_yellow(frame, debug=False, show_images=False, 
                                      yellow_lower=[20, 100, 100], yellow_upper=[30, 255, 255],
                                      white_threshold=180):
    """
    양쪽 노란선 사이의 흰색선을 검출하고 중심 좌표를 반환하는 함수
    
    Parameters:
    frame: 입력 프레임 (BGR)
    debug: 디버그 모드 (상세 로그 출력)
    show_images: 중간 처리 이미지 표시
    yellow_lower: 노란색 하한 HSV 값 [H, S, V]
    yellow_upper: 노란색 상한 HSV 값 [H, S, V]
    white_threshold: 흰색 임계값 (기본값: 180, 조명 환경에 따라 150-220 조정)
    
    Returns:
    line_center_x: 라인 중심 X 좌표 (없으면 None)
    mask: 검출된 마스크
    roi: ROI 영역
    debug_info: 디버그 정보 딕셔너리 (debug=True일 때)
    """
    debug_info = {}
    
    # 1단계: ROI 설정
    h, w = frame.shape[:2]
    roi_y1 = int(h * 0.60)
    roi_y2 = h
    roi_x1 = int(w * 0.10)
    roi_x2 = int(w * 0.75)
    
    roi = frame[roi_y1:roi_y2, roi_x1:roi_x2]
    roi_h, roi_w = roi.shape[:2]
    roi_center_x = roi_w // 2
    
    if debug:
        debug_info['roi'] = {
            'frame_size': (h, w),
            'roi_bounds': (roi_x1, roi_y1, roi_x2, roi_y2),
            'roi_size': (roi_h, roi_w)
        }
        print(f"[1단계] ROI 설정: 프레임 크기={h}x{w}, ROI={roi_x1},{roi_y1}~{roi_x2},{roi_y2}, ROI 크기={roi_h}x{roi_w}")
    
    # 2단계: HSV 변환
    hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    
    # 3단계: 노란색 마스크 생성
    yellow_lower_np = np.array(yellow_lower, dtype=np.uint8)
    yellow_upper_np = np.array(yellow_upper, dtype=np.uint8)
    yellow_mask = cv2.inRange(hsv_roi, yellow_lower_np, yellow_upper_np)
    
    yellow_count = np.sum(yellow_mask == 255)
    yellow_percentage = (yellow_count / (roi_h * roi_w)) * 100
    
    if debug:
        debug_info['yellow'] = {
            'pixel_count': yellow_count,
            'percentage': yellow_percentage,
            'lower': yellow_lower,
            'upper': yellow_upper
        }
        print(f"[2단계] 노란색 검출: 노란색 픽셀={yellow_count}개 ({yellow_percentage:.2f}%)")
    
    if show_images:
        import matplotlib.pyplot as plt
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 4, 1)
        plt.imshow(cv2.cvtColor(roi, cv2.COLOR_BGR2RGB))
        plt.title('ROI')
        plt.axis('off')
        plt.subplot(1, 4, 2)
        plt.imshow(yellow_mask, cmap='gray')
        plt.title('Yellow Mask')
        plt.axis('off')
    
    # 4단계: 양쪽 노란선 위치 찾기
    left_yellow_x = None
    right_yellow_x = None
    
    if yellow_count > 100:  # 최소한의 노란색 픽셀이 있어야 함
        # ROI를 좌우로 나누어 노란선 검출
        left_half = yellow_mask[:, :roi_center_x]
        right_half = yellow_mask[:, roi_center_x:]
        
        # 좌측 노란선 (왼쪽 절반에서 검출)
        left_ys, left_xs = np.where(left_half == 255)
        if len(left_xs) > 50:  # 충분한 픽셀이 있어야 함
            left_yellow_x = int(np.mean(left_xs))
            if debug:
                print(f"[3단계] 좌측 노란선 검출: X={left_yellow_x} (ROI 좌표계)")
        
        # 우측 노란선 (오른쪽 절반에서 검출)
        right_ys, right_xs = np.where(right_half == 255)
        if len(right_xs) > 50:  # 충분한 픽셀이 있어야 함
            right_yellow_x = int(np.mean(right_xs)) + roi_center_x  # ROI 좌표계로 변환
            if debug:
                print(f"[3단계] 우측 노란선 검출: X={right_yellow_x} (ROI 좌표계)")
    
    # 5단계: 검색 영역 정의 (양쪽 노란선 사이)
    margin = 10  # 여유 공간
    search_x1 = 0
    search_x2 = roi_w
    
    if left_yellow_x is not None and right_yellow_x is not None:
        # 양쪽 노란선 사이의 영역
        search_x1 = left_yellow_x + margin
        search_x2 = right_yellow_x - margin
        
        if search_x2 <= search_x1:
            # 영역이 너무 좁으면 기본 ROI 사용
            search_x1 = 0
            search_x2 = roi_w
            if debug:
                print(f"[4단계] 노란선 사이 영역이 너무 좁음, 전체 ROI 사용")
    elif left_yellow_x is not None:
        # 좌측 노란선만 검출된 경우: 노란선 오른쪽부터 검색
        search_x1 = left_yellow_x + margin
        if debug:
            print(f"[4단계] 좌측 노란선만 검출, 오른쪽 영역 검색")
    elif right_yellow_x is not None:
        # 우측 노란선만 검출된 경우: 노란선 왼쪽까지 검색
        search_x2 = right_yellow_x - margin
        if debug:
            print(f"[4단계] 우측 노란선만 검출, 왼쪽 영역 검색")
    else:
        # 노란선이 검출되지 않음: 전체 ROI 영역 사용
        if debug:
            print(f"[4단계] 노란선 미검출, 전체 ROI 영역 사용")
    
    if debug:
        debug_info['search_area'] = {
            'left_yellow': left_yellow_x,
            'right_yellow': right_yellow_x,
            'search_x1': search_x1,
            'search_x2': search_x2
        }
        print(f"[4단계] 검색 영역: X={search_x1}~{search_x2} (ROI 좌표계)")
    
    # 검색 영역이 유효한지 확인
    if search_x2 <= search_x1 or search_x1 < 0 or search_x2 > roi_w:
        search_x1 = 0
        search_x2 = roi_w
    
    # 6단계: 검색 영역 내에서 흰색선 검출
    search_roi = roi[:, search_x1:search_x2]
    
    # 그레이스케일 변환
    gray = cv2.cvtColor(search_roi, cv2.COLOR_BGR2GRAY)
    gray_blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # 흰색선 검출
    _, white_mask = cv2.threshold(gray_blurred, white_threshold, 255, cv2.THRESH_BINARY)
    
    white_count = np.sum(white_mask == 255)
    search_area_size = (search_x2 - search_x1) * roi_h
    white_percentage = (white_count / search_area_size * 100) if search_area_size > 0 else 0
    
    if debug:
        debug_info['white'] = {
            'pixel_count': white_count,
            'percentage': white_percentage,
            'threshold': white_threshold,
            'search_area_size': search_area_size
        }
        print(f"[5단계] 흰색선 검출: 흰색 픽셀={white_count}개 ({white_percentage:.2f}%), 임계값={white_threshold}")
    
    if show_images:
        plt.subplot(1, 4, 3)
        plt.imshow(gray_blurred, cmap='gray')
        plt.title('Gray + Blur')
        plt.axis('off')
        plt.subplot(1, 4, 4)
        plt.imshow(white_mask, cmap='gray')
        plt.title('White Mask')
        plt.axis('off')
        plt.tight_layout()
        plt.show()
    
    # 7단계: 흰색 픽셀 위치 찾기 및 교차로 감지
    ys, xs = np.where(white_mask == 255)
    
    if len(xs) > 0:
        # 슬라이딩 윈도우 기반 중심 계산 (곡선 구간에서 더 안정적인 중심 추정)
        sw_center_x_relative, sw_debug = sliding_window_center(
            white_mask,
            num_windows=8,
            margin=15,
            min_pixels=30,
            debug=debug
        )

        # 슬라이딩 윈도우가 실패하면 단순 평균으로 폴백
        if sw_center_x_relative is None:
            white_center_x_relative = int(np.mean(xs))
            center_mode = 'mean_fallback'
        else:
            white_center_x_relative = sw_center_x_relative
            center_mode = 'sliding_window'

        white_center_y_relative = int(np.mean(ys))
        white_std_x = np.std(xs)
        
        # X 범위 계산 (교차로 감지용)
        x_min = int(np.min(xs))
        x_max = int(np.max(xs))
        x_range = x_max - x_min
        search_area_width = search_x2 - search_x1
        
        # 교차로 감지: 흰색 픽셀이 넓게 분포하면 교차로로 판단
        # X 범위가 검색 영역의 60% 이상이면 교차로
        is_intersection = (x_range > search_area_width * 0.6) and (white_count > search_area_size * 0.3)
        
        # ROI 좌표계로 변환
        white_center_x_roi = search_x1 + white_center_x_relative
        # 전체 프레임 좌표계로 변환
        white_center_x_frame = roi_x1 + white_center_x_roi
        white_center_y_frame = roi_y1 + white_center_y_relative
        
        if debug:
            debug_info['detection'] = {
                'found': True,
                'center_relative': (white_center_x_relative, white_center_y_relative),
                'center_roi': (white_center_x_roi, white_center_y_relative),
                'center_frame': (white_center_x_frame, white_center_y_frame),
                'std_x': white_std_x,
                'pixel_count': len(xs),
                'x_range': (x_min, x_max),
                'x_range_width': x_range,
                'y_range': (int(np.min(ys)), int(np.max(ys))),
                'is_intersection': is_intersection,
                'center_mode': center_mode,
                'sliding_window': sw_debug,
            }
            print(f"[6단계] 라인 검출 성공 ({center_mode}):")
            print(f"  - 검색 영역 내 중심: ({white_center_x_relative}, {white_center_y_relative})")
            print(f"  - ROI 좌표계: 중심=({white_center_x_roi}, {white_center_y_relative}), X 표준편차={white_std_x:.1f}")
            print(f"  - 프레임 좌표계: 중심=({white_center_x_frame}, {white_center_y_frame})")
            print(f"  - 픽셀 수: {len(xs)}개, X 범위: {x_range}픽셀")
            if is_intersection:
                print(f"  - [교차로 감지] 흰색선이 넓게 분포함 (X 범위: {x_range}/{search_area_width})")
        
        if debug or DETECT_WHITE_BETWEEN_YELLOW_DEBUG:
            return white_center_x_frame, white_mask, roi, debug_info, is_intersection
        else:
            return white_center_x_frame, white_mask, roi, is_intersection
    else:
        if debug:
            debug_info['detection'] = {
                'found': False,
                'reason': 'No white pixels detected in search area'
            }
            print(f"[6단계] 라인 검출 실패: 검색 영역 내 흰색 픽셀이 없습니다")
        
        if debug or DETECT_WHITE_BETWEEN_YELLOW_DEBUG:
            return None, white_mask, roi, debug_info, False
        else:
            return None, white_mask, roi, False

print("노란선 사이 흰색선 검출 함수 초기화 완료")
print("디버그 모드: detect_white_line_between_yellow(frame, debug=True, show_images=True)")
print("노란색 범위 조정: yellow_lower=[20, 100, 100], yellow_upper=[30, 255, 255]")
print("흰색 임계값 조정: white_threshold=180 (조명 환경에 따라 150-220)")


In [ ]:
# 슬라이딩 윈도우 기반 라인 중심 계산 함수
# 흰색 이진 마스크(0/255)를 입력으로 받아 곡선 구간에서도 보다 안정적으로 중심 X를 계산합니다.

def sliding_window_center(binary_mask, num_windows=8, margin=15, min_pixels=30, debug=False):
    """ 
    세로 방향 슬라이딩 윈도우로 라인의 x 중심을 계산하는 함수

    Parameters:
    - binary_mask: 2D numpy 배열, 흰색(255)이 라인 픽셀
    - num_windows: 세로 방향 윈도우 개수
    - margin: 이전 윈도우 중심에서 좌우 탐색 범위(px)
    - min_pixels: 윈도우 안에서 라인으로 인정할 최소 픽셀 수
    - debug: 디버그 정보 반환 여부

    Returns:
    - center_x: 전체 마스크 기준 라인 중심 X (없으면 None)
    - debug_info: 디버그용 정보 딕셔너리
    """
    h, w = binary_mask.shape[:2]
    window_height = max(1, h // num_windows)

    # 흰색 픽셀 좌표
    nonzero_y, nonzero_x = np.where(binary_mask == 255)
    if len(nonzero_x) == 0:
        return None, {"reason": "no_white_pixels"} if debug else None

    # 바닥 쪽 히스토그램으로 초기 x 위치 추정
    bottom_y_thresh = int(h * 0.75)
    bottom_inds = nonzero_y >= bottom_y_thresh
    if np.any(bottom_inds):
        base_x = int(np.mean(nonzero_x[bottom_inds]))
    else:
        base_x = int(np.mean(nonzero_x))

    current_x = base_x
    centers = []

    for window_idx in range(num_windows):
        win_y_low = h - (window_idx + 1) * window_height
        win_y_high = h - window_idx * window_height
        if win_y_low < 0:
            win_y_low = 0
        if win_y_high <= win_y_low:
            continue

        # 해당 윈도우 영역의 픽셀 선택
        win_inds = (nonzero_y >= win_y_low) & (nonzero_y < win_y_high)
        win_x = nonzero_x[win_inds]

        if len(win_x) == 0:
            continue

        # 이전 중심 주변만 탐색 (곡선에서도 연속성 유지)
        x_min = max(0, current_x - margin)
        x_max = min(w - 1, current_x + margin)

        lane_inds = (win_x >= x_min) & (win_x <= x_max)
        if np.sum(lane_inds) < min_pixels:
            # 충분한 픽셀이 없으면 이 윈도우는 스킵
            continue

        current_x = int(np.mean(win_x[lane_inds]))
        centers.append(current_x)

    if len(centers) == 0:
        # 슬라이딩 윈도우가 실패하면 단순 평균으로 폴백
        fallback_x = int(np.mean(nonzero_x))
        return (fallback_x, {"mode": "fallback_mean", "base_x": base_x}) if debug else (fallback_x, None)

    center_x = int(np.mean(centers))

    if debug:
        debug_info = {
            "mode": "sliding_window",
            "base_x": base_x,
            "centers": centers,
            "num_windows": num_windows,
            "margin": margin,
            "min_pixels": min_pixels,
        }
        return center_x, debug_info

    return center_x, None

print("슬라이딩 윈도우 라인 중심 함수(sliding_window_center) 초기화 완료")


**2-2. 기존 흰색선 검출 함수 (백업)**<br>


In [ ]:
# 기존 흰색선 검출 함수 (노란선이 없을 때 사용)
DETECT_WHITE_LINE_DEBUG = False  # True로 설정하면 상세 로그 출력
DETECT_WHITE_LINE_SHOW_IMAGES = False  # True로 설정하면 중간 이미지 표시

def detect_white_line(frame, debug=False, show_images=False, threshold=180):
    """
    흰색선을 검출하고 중심 좌표를 반환하는 함수
    
    Parameters:
    frame: 입력 프레임 (BGR)
    debug: 디버그 모드 (상세 로그 출력)
    show_images: 중간 처리 이미지 표시
    threshold: 흰색 임계값 (기본값: 180, 조명 환경에 따라 150-220 조정)
    
    Returns:
    line_center_x: 라인 중심 X 좌표 (없으면 None)
    mask: 검출된 마스크
    roi: ROI 영역
    debug_info: 디버그 정보 딕셔너리 (debug=True일 때)
    """
    debug_info = {}
    
    # 1단계: ROI 설정
    h, w = frame.shape[:2]
    roi_y1 = int(h * 0.60)
    roi_y2 = h
    roi_x1 = int(w * 0.10)
    roi_x2 = int(w * 0.75)
    
    roi = frame[roi_y1:roi_y2, roi_x1:roi_x2]
    roi_h, roi_w = roi.shape[:2]
    
    if debug:
        debug_info['roi'] = {
            'frame_size': (h, w),
            'roi_bounds': (roi_x1, roi_y1, roi_x2, roi_y2),
            'roi_size': (roi_h, roi_w)
        }
        print(f"[1단계] ROI 설정: 프레임 크기={h}x{w}, ROI={roi_x1},{roi_y1}~{roi_x2},{roi_y2}, ROI 크기={roi_h}x{roi_w}")
    
    # 2단계: 그레이스케일 변환
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray_mean = np.mean(gray)
    gray_std = np.std(gray)
    
    if debug:
        debug_info['gray'] = {
            'mean': gray_mean,
            'std': gray_std,
            'min': int(np.min(gray)),
            'max': int(np.max(gray))
        }
        print(f"[2단계] 그레이스케일 변환: 평균={gray_mean:.1f}, 표준편차={gray_std:.1f}, 범위=[{int(np.min(gray))}, {int(np.max(gray))}]")
    
    if show_images:
        import matplotlib.pyplot as plt
        plt.figure(figsize=(10, 3))
        plt.subplot(1, 3, 1)
        plt.imshow(cv2.cvtColor(roi, cv2.COLOR_BGR2RGB))
        plt.title('ROI')
        plt.axis('off')
    
    # 3단계: 가우시안 블러 (노이즈 제거)
    gray_blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    if debug:
        blurred_mean = np.mean(gray_blurred)
        print(f"[3단계] 가우시안 블러: 평균={blurred_mean:.1f} (변화: {blurred_mean-gray_mean:.1f})")
    
    # 4단계: 흰색선 검출 (임계값: 180 이상)
    # 흰색은 밝기가 높은 색상이므로 단일 임계값으로 검출
    _, mask = cv2.threshold(gray_blurred, threshold, 255, cv2.THRESH_BINARY)
    
    if debug:
        white_count = np.sum(mask == 255)
        debug_info['threshold'] = {
            'threshold': threshold,
            'white_pixels': white_count
        }
        print(f"[4단계] 임계값 처리: 흰색 임계값={threshold}, 흰색 픽셀={white_count}")
    
    if show_images:
        plt.subplot(1, 3, 2)
        plt.imshow(gray_blurred, cmap='gray')
        plt.title('Gray + Blur')
        plt.axis('off')
    
    # 5단계: 흰색 마스크 확인
    mask_count = np.sum(mask == 255)
    mask_percentage = (mask_count / (roi_h * roi_w)) * 100
    
    if debug:
        debug_info['mask'] = {
            'pixel_count': mask_count,
            'percentage': mask_percentage
        }
        print(f"[5단계] 흰색 마스크: 흰색 픽셀={mask_count}개 ({mask_percentage:.2f}%)")
    
    if show_images:
        plt.subplot(1, 3, 3)
        plt.imshow(mask, cmap='gray')
        plt.title('Final Mask')
        plt.axis('off')
        plt.tight_layout()
        plt.show()
    
    # 6단계: 흰색 픽셀 위치 찾기
    ys, xs = np.where(mask == 255)
    
    if len(xs) > 0:
        # ROI 좌표계에서의 중심
        line_center_x_roi = int(np.mean(xs))
        line_center_y_roi = int(np.mean(ys))
        line_std_x = np.std(xs)
        
        # 전체 프레임 좌표계로 변환
        line_center_x = roi_x1 + line_center_x_roi
        line_center_y = roi_y1 + line_center_y_roi
        
        if debug:
            debug_info['detection'] = {
                'found': True,
                'center_roi': (line_center_x_roi, line_center_y_roi),
                'center_frame': (line_center_x, line_center_y),
                'std_x': line_std_x,
                'pixel_count': len(xs),
                'x_range': (int(np.min(xs)), int(np.max(xs))),
                'y_range': (int(np.min(ys)), int(np.max(ys)))
            }
            print(f"[6단계] 라인 검출 성공:")
            print(f"  - ROI 좌표계: 중심=({line_center_x_roi}, {line_center_y_roi}), X 표준편차={line_std_x:.1f}")
            print(f"  - 프레임 좌표계: 중심=({line_center_x}, {line_center_y})")
            print(f"  - 픽셀 수: {len(xs)}개")
            print(f"  - X 범위: [{int(np.min(xs))}, {int(np.max(xs))}], Y 범위: [{int(np.min(ys))}, {int(np.max(ys))}]")
        
        if debug or DETECT_WHITE_LINE_DEBUG:
            return line_center_x, mask, roi, debug_info
        else:
            return line_center_x, mask, roi
    else:
        if debug:
            debug_info['detection'] = {
                'found': False,
                'reason': 'No white pixels detected'
            }
            print(f"[6단계] 라인 검출 실패: 흰색 픽셀이 없습니다")
        
        if debug or DETECT_WHITE_LINE_DEBUG:
            return None, mask, roi, debug_info
        else:
            return None, mask, roi

print("흰색선 검출 함수 초기화 완료")
print("디버그 모드: detect_white_line(frame, debug=True, show_images=True, threshold=180)")
print("임계값 조정: 조명이 밝으면 threshold=200, 어두우면 threshold=150으로 설정")


**3. PID 제어 클래스**<br>


In [ ]:
class PIDController:
    """PID 제어기 클래스"""
    
    def __init__(self, kp=0.5, ki=0.0, kd=0.1):
        """
        Parameters:
        kp: 비례 게인 (Proportional gain)
        ki: 적분 게인 (Integral gain)
        kd: 미분 게인 (Derivative gain)
        """
        self.kp = kp
        self.ki = ki
        self.kd = kd
        self.integral = 0
        self.previous_error = 0
    
    def compute(self, error):
        """
        PID 제어 출력 계산
        
        Parameters:
        error: 오차 (라인 중심 - 프레임 중심)
        
        Returns:
        output: 제어 출력값
        """
        # 적분 항 (누적 오차)
        self.integral += error
        
        # 미분 항 (오차 변화율)
        derivative = error - self.previous_error
        
        # PID 출력 계산
        output = (self.kp * error) + (self.ki * self.integral) + (self.kd * derivative)
        
        # 적분 항 제한 (Windup 방지)
        if self.integral > 100:
            self.integral = 100
        elif self.integral < -100:
            self.integral = -100
        
        self.previous_error = error
        
        return output

# PID 제어기 생성 (튜닝 가능)
pid = PIDController(kp=0.8, ki=0.0, kd=0.2)

print("PID 제어기 초기화 완료")


**3-1. 카메라 테스트 (선택사항)**<br>
<br>
**카메라가 제대로 작동하는지 먼저 테스트해보세요**<br>


In [ ]:
# 카메라 테스트 (선택사항)
# 메인 루프 전에 카메라가 제대로 작동하는지 확인

def frame_to_bytes(frame):
    """프레임을 JPEG 바이트로 변환"""
    _, buf = cv2.imencode('.jpg', frame, [int(cv2.IMWRITE_JPEG_QUALITY), 40])
    return buf.tobytes()

if cap is None or not cap.isOpened():
    print("[에러] 카메라가 초기화되지 않았습니다.")
elif video_widget is None:
    print("[에러] 비디오 위젯이 초기화되지 않았습니다. 초기화 셀을 다시 실행하세요.")
else:
    print("카메라 테스트 중...")
    test_count = 0
    max_test = 10
    
    for i in range(max_test):
        ret, test_frame = cap.read()
        if ret:
            test_count += 1
            # 테스트 프레임을 비디오 위젯에 표시
            test_frame_flipped = cv2.flip(test_frame, -1)
            video_widget.value = frame_to_bytes(test_frame_flipped)
            print(f"✓ 프레임 {i+1}/{max_test} 읽기 성공 (크기: {test_frame.shape})")
            time.sleep(0.1)
        else:
            print(f"✗ 프레임 {i+1}/{max_test} 읽기 실패")
            time.sleep(0.1)
    
    if test_count > 0:
        print(f"\n✓ 카메라 정상 작동 확인 ({test_count}/{max_test} 프레임 성공)")
        print("메인 루프 실행 가능")
    else:
        print("\n✗ 카메라 프레임 읽기 실패")
        print("카메라 연결을 확인하세요.")


**4. 흰색선 추종 메인 루프**<br>


In [ ]:
def frame_to_bytes(frame):
    """프레임을 JPEG 바이트로 변환"""
    _, buf = cv2.imencode('.jpg', frame, [int(cv2.IMWRITE_JPEG_QUALITY), 40])
    return buf.tobytes()

# 주행 파라미터
base_speed = 25  # 기본 속도 (0-127)
max_steering = 20  # 최대 조향값
frame_center_x = 160  # 프레임 중앙 X 좌표 (320/2)

# 주행 상태 추적
driving_state = "STRAIGHT"  # STRAIGHT, CURVE, INTERSECTION
error_history = []  # 오차 히스토리 (커브 감지용)
max_history = 10  # 최대 히스토리 길이

print("노란선 사이 흰색선 추종 시작 (Ctrl+C로 종료)")
print(f"기본 속도: {base_speed}, 최대 조향: {max_steering}")
print("주행 모드: 직진/커브/교차로 자동 감지")

# 카메라 및 비디오 위젯 확인
if cap is None or not cap.isOpened():
    print("[에러] 카메라가 초기화되지 않았습니다. 먼저 초기화 셀을 실행하세요.")
elif video_widget is None:
    print("[에러] 비디오 위젯이 초기화되지 않았습니다. 초기화 셀을 다시 실행하세요.")
else:
    print("비디오 위젯 확인 완료")
    try:
        frame_count = 0
        error_count = 0
        max_errors = 10  # 최대 연속 에러 횟수
        
        while True:
            # 카메라 연결 상태 확인
            if not cap.isOpened():
                print("[경고] 카메라 연결이 끊어졌습니다.")
                error_count += 1
                if error_count > max_errors:
                    print("[에러] 카메라 연결 실패. 종료합니다.")
                    break
                time.sleep(0.5)
                continue
            
            ret, frame = cap.read()
            if not ret:
                error_count += 1
                print(f"[경고] 프레임 읽기 실패 ({error_count}/{max_errors})")
                if error_count > max_errors:
                    print("[에러] 연속 프레임 읽기 실패. 종료합니다.")
                    break
                time.sleep(0.1)
                continue
            
            # 프레임을 읽었으면 에러 카운터 리셋
            error_count = 0
            frame_count += 1
            
            # 프레임 상하반전 (카메라 설치 방향에 따라 조정)
            frame = cv2.flip(frame, -1)
            
            # 노란선 사이 흰색선 검출 (디버그 모드 옵션)
            # 매 N번째 프레임마다 디버그 로그 출력
            debug_this_frame = (frame_count % 30 == 0)  # 30프레임마다 디버그
            show_images_this_frame = False  # 이미지는 표시하지 않음 (성능 위해)
            
            # 노란색 HSV 범위 (조정 가능)
            yellow_lower = [20, 100, 100]  # 노란색 하한
            yellow_upper = [30, 255, 255]  # 노란색 상한
            white_threshold = 180  # 흰색 임계값 (조명 환경에 따라 조정: 150-220)
            
            result = detect_white_line_between_yellow(
                frame, 
                debug=debug_this_frame, 
                show_images=show_images_this_frame,
                yellow_lower=yellow_lower,
                yellow_upper=yellow_upper,
                white_threshold=white_threshold
            )
            
            if len(result) == 5:
                line_center_x, mask, roi, debug_info, is_intersection = result
            elif len(result) == 4:
                line_center_x, mask, roi, is_intersection = result
                debug_info = None
            else:
                line_center_x, mask, roi = result
                debug_info = None
                is_intersection = False
            
            if line_center_x is not None:
                # 오차 계산 (라인 중심 - 프레임 중심)
                error = line_center_x - frame_center_x
                
                # 오차 히스토리 업데이트 (커브 감지용)
                error_history.append(error)
                if len(error_history) > max_history:
                    error_history.pop(0)
                
                # 주행 상태 판단
                if is_intersection:
                    # 교차로 감지: 직진 유지
                    driving_state = "INTERSECTION"
                    steering = 0  # 조향 없이 직진
                    if debug_this_frame:
                        print(f"[교차로] 직진 유지 (조향=0)")
                else:
                    # 오차의 절대값이 작으면 직진, 크면 커브
                    abs_error = abs(error)
                    error_avg = np.mean(np.abs(error_history)) if len(error_history) > 0 else abs_error
                    
                    if abs_error < 15 and error_avg < 20:
                        driving_state = "STRAIGHT"
                    else:
                        driving_state = "CURVE"
                    
                    # PID 제어 출력 계산
                    steering = pid.compute(error)
                    
                    
                # 조향값 제한
                steering = np.clip(steering, -max_steering, max_steering)
                
                # 좌우 모터 속도 계산
                left_speed = base_speed + steering
                right_speed = base_speed - steering
                
                # 속도 제한 (0-127)
                left_speed = int(np.clip(left_speed, 0, 127))
                right_speed = int(np.clip(right_speed, 0, 127))
                
                # 모터 제어
                tiki.set_motor_power(tiki.MOTOR_LEFT, left_speed)
                tiki.set_motor_power(tiki.MOTOR_RIGHT, right_speed)
                
                # 디버깅 정보 표시
                cv2.circle(frame, (line_center_x, int(frame.shape[0] * 0.8)), 5, (0, 255, 0), -1)  # 흰색선 중심 (녹색)
                cv2.circle(frame, (frame_center_x, int(frame.shape[0] * 0.8)), 5, (0, 0, 255), -1)  # 프레임 중심 (빨간색)
                cv2.line(frame, (line_center_x, int(frame.shape[0] * 0.8)), 
                         (frame_center_x, int(frame.shape[0] * 0.8)), (255, 0, 0), 2)  # 오차선 (파란색)
                
                # ROI 영역 표시
                h, w = frame.shape[:2]
                roi_y1 = int(h * 0.60)
                roi_y2 = h
                roi_x1 = int(w * 0.10)
                roi_x2 = int(w * 0.75)
                cv2.rectangle(frame, (roi_x1, roi_y1), (roi_x2, roi_y2), (255, 255, 0), 2)  # ROI 영역 (노란색)
                
                # 노란선 위치 표시 (디버그 정보가 있으면)
                if debug_info and 'search_area' in debug_info:
                    sa = debug_info['search_area']
                    if sa['left_yellow'] is not None:
                        left_yellow_frame_x = roi_x1 + sa['left_yellow']
                        cv2.line(frame, (left_yellow_frame_x, roi_y1), (left_yellow_frame_x, roi_y2), (0, 255, 255), 2)  # 좌측 노란선
                    if sa['right_yellow'] is not None:
                        right_yellow_frame_x = roi_x1 + sa['right_yellow']
                        cv2.line(frame, (right_yellow_frame_x, roi_y1), (right_yellow_frame_x, roi_y2), (0, 255, 255), 2)  # 우측 노란선
                    # 검색 영역 표시
                    search_x1_frame = roi_x1 + sa['search_x1']
                    search_x2_frame = roi_x1 + sa['search_x2']
                    cv2.rectangle(frame, (search_x1_frame, roi_y1), (search_x2_frame, roi_y2), (255, 0, 255), 1)  # 검색 영역 (마젠타)
                
                # 정보 텍스트 표시
                state_color = (0, 255, 0) if driving_state == "STRAIGHT" else ((255, 165, 0) if driving_state == "CURVE" else (255, 0, 255))
                cv2.putText(frame, f"State: {driving_state}", (10, 30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, state_color, 2)
                cv2.putText(frame, f"Error: {error:.1f}", (10, 50), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                cv2.putText(frame, f"Steering: {steering:.1f}", (10, 70), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                cv2.putText(frame, f"L:{left_speed} R:{right_speed}", (10, 90), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                
            else:
                # 라인을 찾지 못한 경우 정지
                tiki.stop()
                cv2.putText(frame, "Line not detected - STOP", (10, 30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
            
            # 비디오 위젯 업데이트
            try:
                frame_bytes = frame_to_bytes(frame)
                if frame_bytes is not None and len(frame_bytes) > 0:
                    video_widget.value = frame_bytes
                else:
                    print("[경고] 프레임 바이트 변환 실패")
            except Exception as e:
                print(f"[경고] 비디오 위젯 업데이트 실패: {e}")
            
            # 디버깅: 처음 5프레임 또는 디버그 프레임일 때 정보 출력
            if frame_count <= 5 or debug_this_frame:
                if line_center_x is not None:
                    print(f"[프레임 {frame_count}] 흰색선 검출 성공: 중심={line_center_x}, 오차={line_center_x-frame_center_x:.1f}, 상태={driving_state}")
                    if is_intersection:
                        print(f"  - [교차로] 직진 모드 활성화")
                    if debug_info:
                        if 'yellow' in debug_info:
                            print(f"  - 노란색 픽셀: {debug_info['yellow']['pixel_count']}개 ({debug_info['yellow']['percentage']:.2f}%)")
                        if 'search_area' in debug_info:
                            sa = debug_info['search_area']
                            print(f"  - 좌측 노란선: {sa.get('left_yellow', 'None')}, 우측 노란선: {sa.get('right_yellow', 'None')}")
                        if 'white' in debug_info:
                            print(f"  - 흰색 픽셀: {debug_info['white']['pixel_count']}개 ({debug_info['white']['percentage']:.2f}%)")
                        if 'detection' in debug_info and 'x_range_width' in debug_info['detection']:
                            print(f"  - X 범위: {debug_info['detection']['x_range_width']}픽셀")
                else:
                    print(f"[프레임 {frame_count}] 흰색선 검출 실패")
                    if debug_info and 'yellow' in debug_info:
                        print(f"  - 노란색 픽셀: {debug_info['yellow']['pixel_count']}개")
            
            time.sleep(0.02)  # ~50 FPS

    except KeyboardInterrupt:
        print("\n노란선 사이 흰색선 추종 종료")

    except Exception as e:
        print(f"\n[에러] 예외 발생: {e}")
        print("예외 타입:", type(e).__name__)

    finally:
        # 정지 및 리소스 해제
        print("\n리소스 해제 중...")
        tiki.stop()
        if cap is not None:
            cap.release()
        print("리소스 해제 완료")
        if 'frame_count' in locals():
            print(f"총 처리된 프레임: {frame_count}")


**5. 파라미터 튜닝 가이드**<br>
<br>
**조정 가능한 파라미터:**<br>
- `base_speed`: 기본 주행 속도 (15-30 권장)<br>
- `max_steering`: 최대 조향값 (15-25 권장)<br>
- `PID 파라미터`: kp, ki, kd 값 조정<br>
  - kp 증가 → 빠른 반응, 불안정<br>
  - ki 증가 → 누적 오차 보정, 오버슈트 가능<br>
  - kd 증가 → 진동 억제, 지연 가능<br>
- `threshold`: 흰색선 검출 임계값 (180 기본값, 150-220 조정 가능)<br>
  - 밝은 환경: 200-220<br>
  - 일반 환경: 180<br>
  - 어두운 환경: 150-170<br>
- `ROI 영역`: 검출 영역 조정<br>
<br>
**디버깅 정보:**<br>
- 녹색 원: 흰색선 중심 위치<br>
- 빨간 원: 프레임 중심 위치<br>
- 파란 선: 오차 (흰색선 중심과 프레임 중심 사이)<br>
- 노란 사각형: ROI 영역<br>
- 노란 선: 검출된 노란선 위치 (양쪽)<br>
- 마젠타 사각형: 흰색선 검색 영역 (노란선 사이)<br>
- 텍스트: State (주행 상태), Error, Steering, 좌우 모터 속도<br>
<br>
**주행 상태 표시:**<br>
- STRAIGHT (녹색): 직진 구간<br>
- CURVE (주황색): 커브 구간<br>
- INTERSECTION (마젠타): 교차로 구간 (직진 유지)<br>
